In [1]:
import os
import pandas as pd
from flapjack import Flapjack

In [2]:
fj_url = "localhost:8000" #local
# fj_url = "flapjack.rudge-lab.org:8000" #Web Instance Rudge Lab
#fj_url = "198.59.83.73:8000" #Web Instance Genetic Logic Lab
fj_user = "dylan33smith"
fj_pass = "coco33"

# updated xcel file is in test/test_files
test_file_path = '../../tests/test_files'
excel_path = os.path.join(test_file_path, "flapjack_excel_converter_revamp.xlsx")
'''
hash_map = e2f.flapjack_upload(fj_url, fj_user, fj_pass, excel_path, sbol_hash_map={},
                    add_sbol_uris=False, flapjack_override=True, print_progress=True)
print(hash_map)
'''
confirm=False
overwrite=True

In [59]:
# # log in to flapjack instance
fj = Flapjack(url_base=fj_url) #Local Instance
fj.log_in(username=fj_user, password=fj_pass)

In [22]:
# read in Excel file
xls = pd.ExcelFile(excel_path)
# parse FlapjackCols sheet
fj_conv_sht = xls.parse('FlapjackCols', skiprows=0)
fj_conv_sht

,Sheet Name,ColName,FlapjackName
0,Supplement,Supplement Name,name
1,Supplement,Chemical ID,chemical
2,Supplement,Concentration,concentration
3,Chemical,Chemical Name,name
4,Chemical,Chemical Description,description
5,Chemical,Pubchem ID,pubchemid
6,Vector,Vector Name,name
7,Vector,DNA ID,dna
8,Strain,Strain Name,name
9,Strain,Strain Description,description


In [62]:
# define list of all the sheet names in the XDC excel sheet
types = ['Chemical', 'DNA', 'Supplement', 'Vector', 'Strain', 'Media',
            'Signal', 'Study', 'Assay', 'Sample', 'Measurement', 'Sample Design']

# initiate hashmap for linking to chemicals
    # Linking what to chemicals?
hash_map = {}
# create empty dataframe used to store consolidated data from processed xcel sheets
revamp_df = pd.DataFrame()

# 
for obj in types:
    # Read in the conversion sheet for col name to flapjack name
    fj_conv_sht_obj = fj_conv_sht.loc[(fj_conv_sht['Sheet Name'] == obj)]
    fj_conv_sht_obj = fj_conv_sht_obj.set_index('ColName').to_dict('index')
    # print(fj_conv_sht_obj)

    # read in the object sheet
    obj_df = xls.parse(obj, skiprows=0, index_col=f'{obj} ID')
    cols = list(obj_df.columns)
    # drop columns not used by flapjack and rename the ones that are
    new_cols = []
    col_drop = []
    for col in cols:
        if col in fj_conv_sht_obj.keys():
            new_cols.append(fj_conv_sht_obj[col]['FlapjackName'])
        else:
            col_drop.append(col)

    # print(obj_df)
    # print()
    obj_df = obj_df.drop(columns=col_drop)
    obj_df.columns = new_cols
    obj_df['object'] = obj
    obj_df['flapjackid'] = ''

    revamp_df = revamp_df.append(obj_df)

{'Chemical Name': {'Sheet Name': 'Chemical', 'FlapjackName': 'name'}, 'Chemical Description': {'Sheet Name': 'Chemical', 'FlapjackName': 'description'}, 'Pubchem ID': {'Sheet Name': 'Chemical', 'FlapjackName': 'pubchemid'}}
{'DNA Name': {'Sheet Name': 'DNA', 'FlapjackName': 'name'}}
{'Supplement Name': {'Sheet Name': 'Supplement', 'FlapjackName': 'name'}, 'Chemical ID': {'Sheet Name': 'Supplement', 'FlapjackName': 'chemical'}, 'Concentration': {'Sheet Name': 'Supplement', 'FlapjackName': 'concentration'}}
{'Vector Name': {'Sheet Name': 'Vector', 'FlapjackName': 'name'}, 'DNA ID': {'Sheet Name': 'Vector', 'FlapjackName': 'dna'}}
{'Strain Name': {'Sheet Name': 'Strain', 'FlapjackName': 'name'}, 'Strain Description': {'Sheet Name': 'Strain', 'FlapjackName': 'description'}, 'Strain ID': {'Sheet Name': 'Strain', 'FlapjackName': 'strain'}}
{'Media Name': {'Sheet Name': 'Media', 'FlapjackName': 'name'}, 'Media Description': {'Sheet Name': 'Media', 'FlapjackName': 'description'}}
{'Signal Name

In [54]:
revamp_df

,name,description,pubchemid,object,flapjackid,chemical,concentration,dna,color,DOI,machine,temperature,study,row,col,assay,sampledesign,sample,signal,time,value,media,strain,vector,supplement
Chemical1,ATC,Inducer,54710409.0,Chemical,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chemical2,IPTG,Inducer,656894.0,Chemical,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DNA1,Rep_HIGH_degrate,NaN,NaN,DNA,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DNA2,Rep_LOW_degrate,NaN,NaN,DNA,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DNA3,Reporter,NaN,NaN,DNA,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Supplement1,ATC,NaN,NaN,Supplement,1,Chemical1,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Supplement2,IPTG,NaN,NaN,Supplement,2,Chemical2,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rep_1,Repressilator1,NaN,NaN,Vector,1,NaN,NaN,DNA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rep_2,Repressilator2,NaN,NaN,Vector,2,NaN,NaN,DNA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EcoliT7,E_coli_T7,E.coli T7,NaN,Strain,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
revamp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, Chemical1 to SampleDesign2
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           16 non-null     object 
 1   description    9 non-null      object 
 2   pubchemid      2 non-null      float64
 3   object         62 non-null     object 
 4   flapjackid     62 non-null     object 
 5   chemical       2 non-null      object 
 6   concentration  2 non-null      float64
 7   dna            2 non-null      object 
 8   color          2 non-null      object 
 9   DOI            1 non-null      object 
 10  machine        2 non-null      object 
 11  temperature    2 non-null      float64
 12  study          2 non-null      object 
 13  row            4 non-null      float64
 14  col            4 non-null      float64
 15  assay          4 non-null      object 
 16  sampledesign   4 non-null      object 
 17  sample         40 non-null     object 
 18

# Upload data from excel

If getting an "error no attribute id" change overwrite to False
    
- Otherwise overwrite=overwrite

creating hash_map as you go

In [28]:
#create studys
for index, row in revamp_df[revamp_df['object'] == 'Study'].iterrows():
    flapjack_obj = fj.create(
                            'study',
                            name=row['name'],
                            description=row['description'],
                            confirm=confirm,
                            overwrite=False,
                            )
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]
    # if row[DOI] is not nan patch the DOI
    if not pd.isna(row['DOI']):
        fj.patch('study', flapjack_obj.id[0], doi=row['DOI'])

Returning exististing model, study


In [29]:
#create signals
for index, row in revamp_df[revamp_df['object'] == 'Signal'].iterrows():
    flapjack_obj = fj.create( 
                            'signal',
                            name=row['name'], 
                            description=row['description'],
                            color=row['color'],
                            confirm=confirm,
                            overwrite=False,
                            )
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]

Returning exististing model, signal
Returning exististing model, signal


In [30]:
#create chemicals #todo resolve problem with overwrite True and founding an exitent chemical it return a df
for index, row in revamp_df[revamp_df['object'] == 'Chemical'].iterrows():
    flapjack_obj = fj.create( 
                            'chemical',
                            name=row['name'], 
                            description=row['description'],
                            pubchemid=row['pubchemid'],
                            confirm=confirm,
                            overwrite=False,
                            )
    print(flapjack_obj)
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]
    '''
    if not pd.isna(row['pubchemid']):
        fj.patch('chemical', flapjack_obj.id[0], pubchemid=row['pubchemid'])
    '''

Returning exististing model, chemical
   id name description  pubchemid sboluri
0   1  ATC     Inducer   54710409        
Returning exististing model, chemical
   id  name description  pubchemid sboluri
0   2  IPTG     Inducer     656894        


In [36]:
#create dna
for index, row in revamp_df[revamp_df['object'] == 'DNA'].iterrows():
    flapjack_obj = fj.create( 
                            'dna',
                            name=row['name'], 
                            confirm=confirm,
                            overwrite=False,
                            )
    print(flapjack_obj)
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]

Returning exististing model, dna
   id              name sboluri
0   1  Rep_HIGH_degrate        
Returning exististing model, dna
   id             name sboluri
0   2  Rep_LOW_degrate        
Returning exististing model, dna
   id      name sboluri
0   3  Reporter        


In [37]:
print(hash_map)

{'Study1': 1, 'Signal1': 1, 'Signal9': 2, 'Chemical1': 1, 'Chemical2': 2, 'DNA1': 1, 'DNA2': 2, 'DNA3': 3}


In [38]:
#create medias
for index, row in revamp_df[revamp_df['object'] == 'Media'].iterrows():
    flapjack_obj = fj.create( 
                            'media',
                            name=row['name'], 
                            description=row['description'],
                            confirm=confirm,
                            overwrite=False,
                            )
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]

Returning exististing model, media


In [39]:
#create strains
for index, row in revamp_df[revamp_df['object'] == 'Strain'].iterrows():
    flapjack_obj = fj.create( 
                            'strain',
                            name=row['name'], 
                            description=row['description'],
                            confirm=confirm,
                            overwrite=False,
                            )
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]


Returning exististing model, strain


In [40]:
#create supplements
for index, row in revamp_df[revamp_df['object'] == 'Supplement'].iterrows():
    flapjack_obj = fj.create( 
                            'supplement',
                            name=row['name'], 
                            description=row['description'],
                            chemical=revamp_df.loc[row['chemical'], 'flapjackid'],
                            concentration=row['concentration'],
                            confirm=confirm,
                            overwrite=False,
                            )
    print(flapjack_obj)
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]

Returning exististing model, supplement
   id name  concentration sboluri  chemical
0   1  ATC            0.2                 1
Returning exististing model, supplement
   id  name  concentration sboluri  chemical
0   2  IPTG            0.3                 2


In [41]:
#create vectors
for index, row in revamp_df[revamp_df['object'] == 'Vector'].iterrows():
    flapjack_obj = fj.create( 
                            'vector',
                            name=row['name'], 
                            description=row['description'],
                            dnas=[revamp_df.loc[row['dna'], 'flapjackid']],
                            confirm=confirm,
                            overwrite=False,
                            )
    print(flapjack_obj)
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]

   id            name sboluri dnas
0   1  Repressilator1          [1]
   id            name sboluri dnas
0   2  Repressilator2          [2]


In [42]:
#creaye assays
for index, row in revamp_df[revamp_df['object'] == 'Assay'].iterrows():
    flapjack_obj = fj.create( 
                            'assay',
                            study=revamp_df.loc[row['study'], 'flapjackid'],
                            name=row['name'], 
                            description=row['description'],
                            machine=row['machine'],
                            temperature=row['temperature'],
                            confirm=confirm,
                            overwrite=False,
                            )
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]


In [43]:
# create samples
for index, row in revamp_df[revamp_df['object'] == 'Sample'].iterrows():
    #get flapjack ids from sample design
    media_index = revamp_df.loc[row['sampledesign'], 'media']
    media_id = revamp_df.loc[media_index, 'flapjackid']
    strain_index = revamp_df.loc[row['sampledesign'], 'strain']
    strain_id = revamp_df.loc[strain_index, 'flapjackid']
    vector_index = revamp_df.loc[row['sampledesign'], 'vector']
    vector_id = revamp_df.loc[vector_index, 'flapjackid']
    flapjack_obj = fj.create( 
                            'sample',
                            assay=revamp_df.loc[row['assay'], 'flapjackid'],
                            media=media_id,
                            strain=strain_id,
                            vector=vector_id,
                            row=row['row'],
                            col=row['col'],
                            confirm=confirm,
                            overwrite=False,
                            )
    hash_map[index] = flapjack_obj.id[0]
    revamp_df.loc[index, 'flapjackid'] = flapjack_obj.id[0]
    #TDODO add loop over supplements, or get supplements as list
    if not pd.isna(row['supplement']):
        supplement_index = revamp_df.loc[row['sampledesign'], 'supplement']
        supplement_id = revamp_df.loc[supplement_index, 'flapjackid']
        fj.patch('sample', flapjack_obj.id[0], supplements=supplement_id)
    

Returning exististing model, sample
Returning exististing model, sample


In [44]:
flapjack_obj

,id,sboluri,row,col,assay,media,strain,vector,supplements
0,2,,1,2,2,1,1,2,[]


In [46]:
measurement_df = revamp_df[revamp_df['object'] == 'Measurement'][['time', 'value', 'signal', 'sample']]
unique_samples = set(revamp_df[revamp_df['object'] == 'Measurement']['sample'])
unique_signals = set(revamp_df[revamp_df['object'] == 'Measurement']['signal'])

print(measurement_df)
print(unique_samples)
print(unique_signals)

                time     value   signal   sample
Measurement0    0.00  0.000000  Signal9  Sample1
Measurement1    0.00  0.010331  Signal1  Sample1
Measurement2    0.25  1.953695  Signal9  Sample1
Measurement3    0.25  0.010752  Signal1  Sample1
Measurement4    0.50  1.715337  Signal9  Sample1
Measurement5    0.50  0.011156  Signal1  Sample1
Measurement6    0.75  1.678607  Signal9  Sample1
Measurement7    0.75  0.012010  Signal1  Sample1
Measurement8    1.00  1.587738  Signal9  Sample1
Measurement9    1.00  0.013234  Signal1  Sample1
Measurement10   1.25  1.612557  Signal9  Sample1
Measurement11   1.25  0.016093  Signal1  Sample1
Measurement12   1.50  1.740648  Signal9  Sample1
Measurement13   1.50  0.018807  Signal1  Sample1
Measurement14   1.75  2.094849  Signal9  Sample1
Measurement15   1.75  0.022730  Signal1  Sample1
Measurement16   2.00  2.715779  Signal9  Sample1
Measurement17   2.00  0.027194  Signal1  Sample1
Measurement18   2.25  3.815254  Signal9  Sample1
Measurement19   2.25

In [47]:
for sample in unique_samples:
    sample_df = revamp_df[revamp_df['sample'] == sample]
    sample_id = revamp_df.loc[sample, 'flapjackid']
    print(sample_df, "\n")
    
    '''for signal,data in sample_df.groupby('signal'):
        signal_id = revamp_df.loc[signal, 'flapjackid']
        if signal_id:
            fj.upload_measurements(data, signal=[signal_id], sample=[sample_id])
        time.sleep(1)  # Add a delay of 1 second before uploading the next set of measurements
        '''

              name description  pubchemid       object flapjackid chemical  \
Measurement0   NaN         NaN        NaN  Measurement                 NaN   
Measurement1   NaN         NaN        NaN  Measurement                 NaN   
Measurement2   NaN         NaN        NaN  Measurement                 NaN   
Measurement3   NaN         NaN        NaN  Measurement                 NaN   
Measurement4   NaN         NaN        NaN  Measurement                 NaN   
Measurement5   NaN         NaN        NaN  Measurement                 NaN   
Measurement6   NaN         NaN        NaN  Measurement                 NaN   
Measurement7   NaN         NaN        NaN  Measurement                 NaN   
Measurement8   NaN         NaN        NaN  Measurement                 NaN   
Measurement9   NaN         NaN        NaN  Measurement                 NaN   
Measurement10  NaN         NaN        NaN  Measurement                 NaN   
Measurement11  NaN         NaN        NaN  Measurement          

In [ ]:
for sample in unique_samples:
    sample_df = revamp_df[revamp_df['sample'] == sample]
    sample_id = revamp_df.loc[sample, 'flapjackid']
    for signal,data in sample_df.groupby('signal'):
        signal_id = revamp_df.loc[signal, 'flapjackid']
        if signal_id:
            fj.upload_measurements(data, signal=[signal_id], sample=[sample_id])
        time.sleep(1)  # Add a delay of 1 second before uploading the next set of measurements

In [29]:
# create measurements
unique_samples = set(revamp_df[revamp_df['object'] == 'Measurement']['sample'])
unique_signals = set(revamp_df[revamp_df['object'] == 'Measurement']['signal'])
for sample in unique_samples:
    sample_id = revamp_df.loc[sample, 'flapjackid']
    for signal in unique_signals:
        signal_id = revamp_df.loc[signal, 'flapjackid']
        measurement_df = revamp_df[revamp_df['object'] == 'Measurement'][revamp_df['sample'] == sample][revamp_df['signal'] == signal][['time', 'value']]
        fj.upload_measurements(measurement_df, signal=[signal_id], sample=sample_id)
        time.sleep(2)

/var/folders/65/dt4l3nw13q57n8x9nlly46640000gn/T/ipykernel_74414/2891085989.py:8: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



TypeError: 'numpy.int64' object is not iterable

In [30]:
measurement_df

,time,value
Measurement200,0.00,0.000000
Measurement202,0.25,1.725928
Measurement204,0.50,1.462755
Measurement206,0.75,1.145845
Measurement208,1.00,1.236872
Measurement210,1.25,1.311550
Measurement212,1.50,1.647446
Measurement214,1.75,2.025493
Measurement216,2.00,2.664766
Measurement218,2.25,3.073661


In [28]:
import time
# create measurements
unique_samples = set(revamp_df[revamp_df['object'] == 'Measurement']['sample'])
unique_signals = set(revamp_df[revamp_df['object'] == 'Measurement']['signal'])
for sample in unique_samples:
    sample_id = revamp_df.loc[sample, 'flapjackid']
    for signal in unique_signals:
        signal_id = revamp_df.loc[signal, 'flapjackid']
        measurement_df = revamp_df[(revamp_df['object'] == 'Measurement') & (revamp_df['sample'] == sample) & (revamp_df['signal'] == signal)][['time', 'value']]
        fj.upload_measurements(measurement_df, signal=[signal_id], sample=sample_id)
        time.sleep(10)

TypeError: 'numpy.int64' object is not iterable

In [27]:
# create measurements
unique_samples = set(revamp_df[revamp_df['object'] == 'Measurement']['sample'])
unique_signals = set(revamp_df[revamp_df['object'] == 'Measurement']['signal'])
for sample in unique_samples:
    sample_id = [revamp_df.loc[sample, 'flapjackid']]
    for signal in unique_signals:
        signal_id = [revamp_df.loc[signal, 'flapjackid']]
        measurement_df = revamp_df[(revamp_df['object'] == 'Measurement') & (revamp_df['sample'] == sample) & (revamp_df['signal'] == signal)][['time', 'value']]
        fj.upload_measurements(measurement_df, signal=signal_id, sample=sample_id)


ConnectionClosedError: received 1011 (unexpected error); then sent 1011 (unexpected error)

In [ ]:
# create measurements
unique_samples = set(revamp_df[revamp_df['object'] == 'Measurement']['sample'])
unique_signals = set(revamp_df[revamp_df['object'] == 'Measurement']['signal'])
for sample in unique_samples:
    sample_id = revamp_df.loc[sample, 'flapjackid']
    for signal in unique_signals:
        signal_id = revamp_df.loc[signal, 'flapjackid']
        measurement_df = revamp_df[revamp_df['object'] == 'Measurement'][revamp_df['sample'] == sample][revamp_df['signal'] == signal][['time', 'value']]
        fj.upload_measurements(measurement_df, signal=[signal_id], sample=sample_id)


In [26]:
measurement_df

,time,value
Measurement200,0.00,0.000000
Measurement202,0.25,1.725928
Measurement204,0.50,1.462755
Measurement206,0.75,1.145845
Measurement208,1.00,1.236872
Measurement210,1.25,1.311550
Measurement212,1.50,1.647446
Measurement214,1.75,2.025493
Measurement216,2.00,2.664766
Measurement218,2.25,3.073661


In [63]:
measurement_df = revamp_df[revamp_df['object'] == 'Measurement'][revamp_df['sample'] == sample][revamp_df['signal'] == signal][['time', 'value']]
measurement_df

/var/folders/65/dt4l3nw13q57n8x9nlly46640000gn/T/ipykernel_28112/3758165704.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,time,value
Measurement201,0.00,0.010444
Measurement203,0.25,0.010294
Measurement205,0.50,0.011338
Measurement207,0.75,0.012736
Measurement209,1.00,0.013692
Measurement211,1.25,0.015990
Measurement213,1.50,0.018740
Measurement215,1.75,0.022822
Measurement217,2.00,0.026213
Measurement219,2.25,0.035800


In [ ]:
media_id = revamp_df.loc[media_index, 'flapjackid']

In [61]:
for i in unique_samples:
    print(i)
    sample_id = revamp_df.loc[i, 'flapjackid']
    print(sample_id)

Sample2
21696
Sample1
21695


In [40]:
unique_samples = set(revamp_df[revamp_df['object'] == 'Measurement']['sample'])
unique_samples

{'Sample1', 'Sample2'}

In [51]:
unique_signals = set(revamp_df[revamp_df['object'] == 'Measurement']['signal'])
unique_signals

{'Signal1', 'Signal9'}

In [45]:
# create a dataframe containing only the measurements
measurement_df = revamp_df[revamp_df['object'] == 'Measurement'][revamp_df['sample'] == 'Sample1']
measurement_df


/var/folders/65/dt4l3nw13q57n8x9nlly46640000gn/T/ipykernel_28112/1582080924.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,name,description,pubchemid,object,flapjackid,chemical,concentration,dna,color,DOI,...,assay,sampledesign,sample,signal,time,value,media,strain,vector,supplement
Measurement0,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal9,0.00,0.000000,NaN,NaN,NaN,NaN
Measurement1,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal1,0.00,0.010331,NaN,NaN,NaN,NaN
Measurement2,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal9,0.25,1.953695,NaN,NaN,NaN,NaN
Measurement3,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal1,0.25,0.010752,NaN,NaN,NaN,NaN
Measurement4,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal9,0.50,1.715337,NaN,NaN,NaN,NaN
Measurement5,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal1,0.50,0.011156,NaN,NaN,NaN,NaN
Measurement6,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal9,0.75,1.678607,NaN,NaN,NaN,NaN
Measurement7,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal1,0.75,0.012010,NaN,NaN,NaN,NaN
Measurement8,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal9,1.00,1.587738,NaN,NaN,NaN,NaN
Measurement9,NaN,NaN,NaN,Measurement,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Sample1,Signal1,1.00,0.013234,NaN,NaN,NaN,NaN


In [50]:
#create a DataFrame just with time and value columns
data_df = measurement_df[['time', 'value']]
data_df

,time,value
Measurement0,0.00,0.000000
Measurement1,0.00,0.010331
Measurement2,0.25,1.953695
Measurement3,0.25,0.010752
Measurement4,0.50,1.715337
Measurement5,0.50,0.011156
Measurement6,0.75,1.678607
Measurement7,0.75,0.012010
Measurement8,1.00,1.587738
Measurement9,1.00,0.013234


In [48]:
data_df = measurement_df[measurement_df['signal']], measurement_df['value']
data_df

KeyError: "None of [Index(['Signal9', 'Signal1', 'Signal9', 'Signal1', 'Signal9', 'Signal1',\n       'Signal9', 'Signal1', 'Signal9', 'Signal1', 'Signal9', 'Signal1',\n       'Signal9', 'Signal1', 'Signal9', 'Signal1', 'Signal9', 'Signal1',\n       'Signal9', 'Signal1'],\n      dtype='object')] are in the [columns]"

In [ ]:
revamp_df[revamp_df['object'] == 'Measurement']['sample'][revamp_df['object'] == 'Measurement']['sample']

In [ ]:
#create measurements


In [62]:
revamp_df[revamp_df['object'] == 'Study']

,name,description,pubchemid,object,chemical,concentration,dna,color,DOI,machine,...,study,row,col,assay,media,strain,sample,signal,time,value
Study1,excel2flapjack,e2f test,NaN,Study,NaN,NaN,NaN,NaN,https://doi.org/10.1371/journal.pone.0187163,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# select the chemical name using loc and column name
revamp_df.loc['Chemical1']['name']

'ATC'

In [52]:
revamp_df.loc['Study1']['name']

'excel2flapjack'

In [34]:
# add column to indicate object type
revamp_df['Type'] = revamp_df.index.str.split('_').str[0]
revamp_df


,name,description,pubchemid,chemical,concentration,dna,color,DOI,machine,temperature,...,row,col,assay,media,strain,sample,signal,time,value,Type
Chemical1,ATC,Inducer,Pubchem 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chemical1
Chemical2,IPTG,Inducer,Pubchem 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chemical2
DNA1,Rep_HIGH_degrate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNA1
DNA2,Rep_LOW_degrate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNA2
DNA3,Reporter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNA3
Supplement1,ATC,NaN,NaN,Chemical1,0.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplement1
Supplement2,IPTG,NaN,NaN,Chemical2,0.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Supplement2
Rep_1,Repressilator1,NaN,NaN,NaN,NaN,DNA1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rep
Rep_2,Repressilator2,NaN,NaN,NaN,NaN,DNA2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rep
EcoliT7,E_coli_T7,E.coli T7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EcoliT7


In [ ]:

# order is important as Chemicals and DNA must be created before
# they can be referenced


    # Create a dictionary of the data for flapjack
    obj_dict = obj_df.to_dict('index')

    # Upload all the objects to flapjack
    for key in obj_dict:
        if print_progress:
            print(key)
        data = obj_dict[key]

        if key in sbol_hash_map and add_sbol_uris:
            data['sboluri'] = sbol_hash_map[key]

        # Change to flapjack id rather than name for chemicals and dnas
        lookups = {'chemical', 'dnas', 'study', 'vector', 'strain', 'media', 'assay', 'sample', 'signal'}
        lk_inter = lookups.intersection(set(data.keys()))
        for it in list(lk_inter):
            data[it] = hash_map[data[it]]

        data['model'] = obj.lower()
        print(data)
        flapjack_id = fj.create(**data, confirm=not(flapjack_override))

        # add Chemical and DNA to hash map to allow cross referencing
        hash_map[key] = flapjack_id.id[0]

return hash_map

# Extra things

In [1]:
plate_96_wells = [
'A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12',
'B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11','B12',
'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12',
'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12',
'E1','E2','E3','E4','E5','E6','E7','E8','E9','E10','E11','E12',
'F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12',
'G1','G2','G3','G4','G5','G6','G7','G8','G9','G10','G11','G12',
'H1','H2','H3','H4','H5','H6','H7','H8','H9','H10','H11','H12'
]

In [2]:
for i in range(0,11):
    print(plate_96_wells[i])

A1
A2
A3
A4
A5
A6
A7
A8
A9
A10
A11
